# Задача

В следующей ячейке написал код по обучению модели машинного обучения для классификации Ирисов.
Необходимо реализовать веб-сервис на Flask, который бы позволял использовать эту модель для классификации через сеть.

In [3]:
from sklearn.datasets import load_iris
import pickle
from sklearn.linear_model import LogisticRegression


X, y = load_iris(return_X_y=True)
clf = LogisticRegression(random_state=0).fit(X, y)

После того, как вы реализуете свой веб-сервис, достаточно будет его запустить и нажать кнопку "Отправить решение". После нажатия автоматически запустится скрипт `check-server.py`, который создаст файл `result.json`. 

Сам скрипт можно использовать для проверки корректности своего решения.

In [4]:
raw_data = pickle.dumps(clf)

with open('fmeter-model.pickle', 'wb') as f:
    f.write(raw_data)

In [98]:
%%writefile server.py
from flask import Flask, request
import json
import pickle
import re

app = Flask(__name__)  # Основной объект приложения Flask


def load_model(pickle_path):
    with open(pickle_path, 'rb') as f:
        raw_data = f.read()
        model = pickle.loads(raw_data)
    return model

model = load_model('fmeter-model.pickle')

@app.route('/')
def hello():
    return "Hello, from Flask"

@app.route('/iris', methods=["GET", "POST"])
def iris_handler():
    if request.method == 'POST':
        data = request.get_json(force=True) 
        result = model.predict([data['iris']])[0]  
        response = {
            "result": int(result)
        }
        return json.dumps(response)
    else:
        return "You should use only POST query"

if __name__ == '__main__':
    app.run("0.0.0.0", 8000)  # Запускаем сервер на 8000 порту

Overwriting server.py


In [99]:
! launch-server.sh server.py

Success!


In [2]:
! cat $(which check-server.py)

#!/usr/bin/env python3

import requests
import json

questions = [
    [4.6, 3.1, 1.5, 0.2],
    [5.2, 2.7, 3.9, 1.4],
    [6.9, 3.1, 5.1, 2.3]
]

result = []
for q in questions:
    data = {
        'iris': q
    }

    r = requests.post("http://localhost:8000/iris", json=data)
    result.append(r.json()['result'])

with open('/home/jovyan/work/result.json', 'w') as f:
    f.write(json.dumps(result, indent=4))

In [105]:
! cat log.txt

 * Serving Flask app "server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:8000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Aug/2021 18:23:26] "POST /iris HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2021 18:23:29] "POST /iris HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2021 18:23:30] "POST /iris HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2021 18:23:48] "POST /iris HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2021 18:23:48] "POST /iris HTTP/1.1" 200 -
127.0.0.1 - - [20/Aug/2021 18:23:48] "POST /iris HTTP/1.1" 200 -
